<a href="https://colab.research.google.com/github/kaitas/ChatGPT-Colab/blob/main/How_to_call_functions_with_chat_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenGPTチャットモデルで関数を呼び出す方法
このノートでは、GPTモデルの機能を拡張するために、チャット補完APIを外部関数と組み合わせて使用する方法について説明します。

[original]
https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb

## functionsの使用方法
functionsはChatCompletion APIのオプションパラメータで、関数の仕様を提供するために使用することができます。この目的は、モデルが関数入力スキーマに準拠した出力を生成することを可能にすることです。なお、APIは実際に関数の呼び出しを実行することはありません。モデルの出力を使って関数呼び出しを実行するのは、開発者次第です。

functionsパラメータが提供されている場合、デフォルトでは、モデルは関数のいずれかを使用することが適切である場合に決定します。また、function_callパラメータに{"name"}を設定することで、特定の関数を使用するようにAPIを強制することもできます： "<insert-function-name>"}とすることで、特定の関数を強制的に使用させることも可能です。関数が使用された場合、出力には "finish_reason "が含まれます： "function_call "と、関数名と生成された関数引数を持つfunction_callオブジェクトがレスポンスに含まれます。

関数は、以下のフィールドで指定します：

- `Name`： Name：関数の名前。
- `Description`： 関数が何をするのかの説明。モデルはこれを用いて、関数を呼び出すタイミングを決定します。
- `Parameters`: パラメータ： parametersオブジェクトは、関数が必要とするすべての入力フィールドを含んでいます。
これらの入力は、次のようなタイプになります： String、Number、Boolean、Object、Null、AnyOf。詳細については、[APIリファレンスドキュメント](https://platform.openai.com/docs/api-reference/chat)を参照してください。
- `Required`（必須）： クエリを作成するために必要なパラメータを指定します。残りはオプションとして扱われます。

関数を実行し、関数実行の出力をアシスタントに直接戻すことで、関数呼び出しを連鎖させることができます。これは、モデルが無限に関数を呼び続ける無限ループの挙動につながる可能性がありますが、これを防ぐためにガードレールを設置することができます。

# ウォークスルー

このクックブックでは、以下のワークフローを説明します：

- 基本的な概念： 基本的なコンセプト：サンプル関数を作成し、必要に応じてAPIにそれを使用させる。
- APIコールを関数実行に統合する： APIコールを使用して関数の引数を生成し、関数を実行するエージェントを作成する。
- 複数の関数を使用する： ユーザーに応答する前に、複数の関数を順番に呼び出すことができるようにする。

In [152]:
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install termcolor
!pip install openai
!pip install requests
!pip install arxiv
!pip install pandas
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [153]:
import arxiv
import ast
import concurrent
from csv import writer
from IPython.display import display, Markdown, Latex
import json
import openai
import os
import pandas as pd
from PyPDF2 import PdfReader
import requests
from scipy import spatial
from tenacity import retry, wait_random_exponential, stop_after_attempt
import tiktoken
from tqdm import tqdm
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"


In [154]:
# 自分はGCPのSecretManagerサービスを使ってカギを保存しています
!pip install google-cloud-secret-manager
from google.colab import auth
auth.authenticate_user()
from google.cloud import secretmanager

def access_secret(project_id, secret_name, version='latest'):
    client = secretmanager.SecretManagerServiceClient()
    name = client.secret_version_path(project_id, secret_name, version)
    response = client.access_secret_version(request={"name":name})
    payload = response.payload.data.decode("UTF-8")
    return payload

openai.api_key = access_secret("awfree-aki2021", "OPENAI_API_KEY")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## ユーティリティ
まず、Chat Completions APIを呼び出したり、会話の状態を維持・管理するためのユーティリティをいくつか定義しておきましょう。

In [155]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [156]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

## 基本的な考え方
次に、`get_current_weather`という関数の仕様書を作成します。後で、この関数仕様をAPIに渡して、仕様に沿った関数引数を生成するようにします。

In [157]:
functions = [
    {
        "name": "get_current_weather",
        "description": "現在の天気を取得する",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "都市と州（例：San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "使用する温度の単位です。ユーザーの位置情報から推測されます。",
                },
            },
            "required": ["location", "format"],
        },
    }
]

In [158]:
conversation = Conversation()
conversation.add_message("user", "きょうはどんな天気？")

In [159]:
# モデルはまず、天気予報の機能を使うために必要な情報をユーザーに要求する
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
conversation.add_message(assistant_message["role"], assistant_message["content"])
assistant_message

{'role': 'assistant', 'content': 'どこの地域の天気情報が知りたいですか？'}

In [160]:
# ユーザーが必要な情報を提供すると、モデルは関数の引数を生成することができます。
conversation.add_message("user", "日本の横浜に住んでます")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions
)
chat_response.json()["choices"][0]

{'index': 0,
 'message': {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'get_current_weather',
   'arguments': '{\n  "location": "Yokohama, Japan",\n  "format": "celsius"\n}'}},
 'finish_reason': 'function_call'}

↑関数 `get_current_weather`のファンクションコールと引数を表現するJSONが生成されました。

★ここまでJSONで出力せよとは指定していないことに注目。

## APIコールと関数実行を統合する
次の例では、入力がモデルで生成された関数を実行する方法を示し、これを使用して、データベースに関する質問に答えるエージェントを実装します。簡単のために、Chinookサンプルデータベースを使用します。

注：SQL生成のユースケースは、本番環境ではリスクが高いです - モデルは一貫したSQL構文を生成する際に信頼できないことがあります。この問題を解決する、より信頼性の高い方法は、モデルから目的のカラムを入力として受け取るクエリ生成APIを構築することかもしれません。

### SQLデータベース情報を引き出す
まず、SQLiteデータベースからデータを抽出するための便利なユーティリティ関数をいくつか定義しましょう。

In [161]:
import sqlite3
# 公式からDBファイルを入手 https://github.com/openai/openai-cookbook/blob/main/examples/data/Chinook.db
conn = sqlite3.connect("sample_data/Chinook.db")
print("データベースを正常に開きました")

データベースを正常に開きました


In [162]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

これらのユーティリティ関数を使用して、データベーススキーマの表現を抽出することができます。


In [163]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)


前回と同様に、APIに引数を生成させる関数の仕様を定義します。データベース・スキーマを関数仕様に挿入していることに注目してください。これは、モデルが知っておくべき重要なことです。

In [164]:
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required": ["query"],
        },
    }
]

## SQLの実行
では、エージェントがデータベースへの問い合わせに使用する関数を実装しましょう。また、Chat Completions APIへの呼び出しと、それが呼び出される関数を統合するためのユーティリティを実装する必要があります。


In [184]:
def ask_database(conn, query):
    """指定されたSQLクエリでSQLiteデータベースに問い合わせを行う関数です。"""
    try:
        results = conn.execute(query).fetchall()
        return results
    except Exception as e:
        raise Exception(f"SQL error: {e}")


def chat_completion_with_function_execution(messages, functions=None):
    """この関数は、ChatCompletion APIコールを行い、関数コールが要求された場合は、その関数を実行する。"""
    try:
        response = chat_completion_request(messages, functions)
        full_message = response.json()["choices"][0]
        if full_message["finish_reason"] == "function_call":
            print(f"Function generation requested, calling function")
            return call_function(messages, full_message)
        else:
            print(f"Function not required, responding to user")
            return response.json()
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return response


def call_function(messages, full_message):
    """モデルで生成された関数引数を用いて、関数呼び出しを実行する。"""

    # We'll add our one function here - this can be extended with any additional functions
    if full_message["message"]["function_call"]["name"] == "ask_database":
        query = eval(full_message["message"]["function_call"]["arguments"])
        print(f"Prepped query is {query}")
        try:
            results = ask_database(conn, query["query"])
        except Exception as e:
            print(e)

            # この次のブロックは、後続の呼び出しでクエリ生成の問題を修正しようとするものです。
            messages.append(
                {
                    "role": "system",
                    "content": f"""Query: {query['query']}
The previous query received the error {e}.
Please return a fixed SQL query in plain text.
Your response should consist of ONLY the SQL query with the separator sql_start at the beginning and sql_end at the end""",
                }
            )
            response = chat_completion_request(messages, model="gpt-4-0613")

            # 修正したSQLクエリで再試行する。2回目に失敗した場合は終了します。
            try:
                cleaned_query = response.json()["choices"][0]["message"][
                    "content"
                ].split("sql_start")[1]
                cleaned_query = cleaned_query.split("sql_end")[0]
                print(cleaned_query)
                results = ask_database(conn, cleaned_query)
                print(results)
                print("Got on second try")

            except Exception as e:
                print("Second failure, exiting")

                print(f"Function execution failed")
                print(f"Error message: {e}")

        messages.append(
            {"role": "function", "name": "ask_database", "content": str(results)}
        )

        try:
            response = chat_completion_request(messages)
            return response.json()
        except Exception as e:
            print(type(e))
            print(e)
            raise Exception("ファンクションチャットのリクエストに失敗しました")
    else:
        raise Exception("機能が存在しないため、呼び出すことができない")


In [185]:
agent_system_message = """あなたはChinookGPT、Chinook Music Databaseからユーザーの質問に対する回答を得る、親切なアシスタントです。
できるだけ多くの情報をユーザーに提供すること。
さあ開始！"""

sql_conversation = Conversation()
sql_conversation.add_message("system", agent_system_message)
sql_conversation.add_message(
    "user", "こんにちは、楽曲数の多いアーティストトップ5は誰ですか？"
)


In [186]:
chat_response = chat_completion_with_function_execution(
    sql_conversation.conversation_history, functions=functions
)
try:
    assistant_message = chat_response["choices"][0]["message"]["content"]
    print(assistant_message)
except Exception as e:
    print(e)
    print(chat_response)

Function generation requested, calling function
Prepped query is {'query': 'SELECT Artist.Name, COUNT(Track.TrackId) AS TotalTracks FROM Artist JOIN Album ON Artist.ArtistId = Album.ArtistId JOIN Track ON Album.AlbumId = Track.AlbumId GROUP BY Artist.ArtistId ORDER BY TotalTracks DESC LIMIT 5'}
楽曲数の多いアーティストトップ5は以下の通りです：

1. Iron Maiden - 213曲
2. U2 - 135曲
3. Led Zeppelin - 114曲
4. Metallica - 112曲
5. Lost - 92曲

これらのアーティストは、Chinook Music Databaseにおいて最も多くの楽曲を持っています。


In [187]:
sql_conversation.add_message("assistant", assistant_message)
sql_conversation.display_conversation(detailed=True)

system: あなたはChinookGPT、Chinook Music Databaseからユーザーの質問に対する回答を得る、親切なアシスタントです。
できるだけ多くの情報をユーザーに提供すること。
さあ開始！


user: こんにちは、楽曲数の多いアーティストトップ5は誰ですか？


function: [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]


assistant: 楽曲数の多いアーティストトップ5は以下の通りです：

1. Iron Maiden - 213曲
2. U2 - 135曲
3. Led Zeppelin - 114曲
4. Metallica - 112曲
5. Lost - 92曲

これらのアーティストは、Chinook Music Databaseにおいて最も多くの楽曲を持っています。




In [188]:
sql_conversation.add_message(
    "user", "最も多くの楽曲が収録されているアルバム名を教えてください。"
)

In [189]:
chat_response = chat_completion_with_function_execution(
    sql_conversation.conversation_history, functions=functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
assistant_message

Function generation requested, calling function
Prepped query is {'query': 'SELECT Album.Title, COUNT(Track.TrackId) AS NumTracks FROM Album JOIN Track ON Album.AlbumId = Track.AlbumId GROUP BY Album.Title ORDER BY NumTracks DESC LIMIT 1;'}


'Chinook Music Databaseにおいて、最も多くの楽曲が収録されているアルバムは「Greatest Hits」という名前のアルバムです。このアルバムには57曲が収録されています。'

In [191]:
sql_conversation.add_message("assistant", assistant_message)

In [192]:
sql_conversation.display_conversation(detailed=True)
# 実際にはPython上では色分けされているはずです

system: あなたはChinookGPT、Chinook Music Databaseからユーザーの質問に対する回答を得る、親切なアシスタントです。
できるだけ多くの情報をユーザーに提供すること。
さあ開始！


user: こんにちは、楽曲数の多いアーティストトップ5は誰ですか？


function: [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]


assistant: 楽曲数の多いアーティストトップ5は以下の通りです：

1. Iron Maiden - 213曲
2. U2 - 135曲
3. Led Zeppelin - 114曲
4. Metallica - 112曲
5. Lost - 92曲

これらのアーティストは、Chinook Music Databaseにおいて最も多くの楽曲を持っています。


user: 最も多くの楽曲が収録されているアルバム名を教えてください。


function: [('Greatest Hits', 57)]


assistant: Chinook Music Databaseにおいて、最も多くの楽曲が収録されているアルバムは「Greatest Hits」という名前のアルバムです。このアルバムには57曲が収録されています。


assistant: Chinook Music Databaseにおいて、最も多くの楽曲が収録されているアルバムは「Greatest Hits」という名前のアルバムです。このアルバムには57曲が収録されています。




## 複数の関数の使用

では、モデルに複数の関数を呼び出して提供するシナリオを構築してみましょう。ここでは、arXivのデータを使って、学術的なテーマに関する質問に答えるエージェントを作成することにします。このエージェントは、2つの新しい関数を自由に使うことができます：

- `get_articles`： get_articles：あるテーマに関するarXivの論文を取得し、リンク付きでユーザーのために要約する関数です。
- `read_article_and_summarize`： この関数は、以前に検索された論文の1つを取り、その全体を読み、核となる議論、証拠、結論を要約する。
これにより、複数のサービスから選択できる多機能ワークフローに慣れることができ、最初の機能からのデータの一部が2番目の機能で使用されるように永続化されることになります。

arXiv検索
まず、2つの機能を支えるユーティリティをいくつか設定します。

ダウンロードした論文は、ディレクトリ（ここでは ./data/papers を使用）に保存されます。ダウンロードした論文の埋め込みと詳細を保存するために、arxiv_library.csvというファイルを作成し、summarize_textを使って検索するようにします。


In [193]:
#@title mount to drive (ドライブにマウント)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [194]:
# ダウンロードした論文を保存するディレクトリを設定する
# 事前に該当のフォルダ（papersまで）作っておいてください
data_dir = os.path.join(os.curdir, "/content/drive/MyDrive/Colab Notebooks/arXiv", "papers")
paper_dir_filepath = "/content/drive/MyDrive/Colab Notebooks/arXiv/arxiv_library.csv"

# ダウンロードしたファイルを格納するための空白のデータフレームを生成する。
df = pd.DataFrame(list())
df.to_csv(paper_dir_filepath)

In [195]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def embedding_request(text):
    response = openai.Embedding.create(input=text, model=EMBEDDING_MODEL)
    return response


def get_articles(query, library=paper_dir_filepath, top_k=5):
    """この関数は、ユーザのクエリに基づいて、関連性の高い順に並べられた top_k の記事を取得します。
    また、read_article_and_summarizeで取得できるように、ファイルをダウンロードしてarxiv_library.csvに保存します。
    """
    search = arxiv.Search(
        query=query, max_results=top_k, sort_by=arxiv.SortCriterion.Relevance
    )
    result_list = []
    for result in search.results():
        result_dict = {}
        result_dict.update({"title": result.title})
        result_dict.update({"summary": result.summary})

        # 最初に提供されたurlを取る
        result_dict.update({"article_url": [x.href for x in result.links][0]})
        result_dict.update({"pdf_url": [x.href for x in result.links][1]})
        result_list.append(result_dict)
        print("pdf_url:" + [x.href for x in result.links][1])

        # ライブラリファイルにリファレンスを保存する
        response = embedding_request(text=result.title)
        file_reference = [
            result.title,
            result.download_pdf(data_dir),
            response["data"][0]["embedding"],
        ]

        # ファイルへの書き込み
        with open(library, "a") as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(file_reference)
            f_object.close()
    return result_list

In [197]:
# 検索が機能していることをテストする
## result_output = get_articles("ppo reinforcement learning")
result_output = get_articles("computer graphics metaverse")
result_output[0]

pdf_url:http://arxiv.org/pdf/2205.02764v1
pdf_url:http://arxiv.org/pdf/2305.11911v1
pdf_url:http://arxiv.org/pdf/2304.13931v1
pdf_url:http://arxiv.org/pdf/2208.00369v1
pdf_url:http://arxiv.org/pdf/2303.10289v1


{'title': 'Edge-enabled Metaverse: The Convergence of Metaverse and Mobile Edge Computing',
 'summary': 'The Metaverse is a virtual environment where users are represented by avatars\nto navigate a virtual world, which has strong links with the physical one.\nState-of-the-art Metaverse architectures rely on a cloud-based approach for\navatar physics emulation and graphics rendering computation. Such centralized\ndesign is unfavorable as it suffers from several drawbacks caused by the long\nlatency required for cloud access, such as low quality visualization. To solve\nthis issue, in this paper, we propose a Fog-Edge hybrid computing architecture\nfor Metaverse applications that leverage an edge-enabled distributed computing\nparadigm, which makes use of edge devices computing power to fulfil the\nrequired computational cost for heavy tasks such as collision detection in\nvirtual universe and computation of 3D physics in virtual simulation. The\ncomputational cost related to an entity i

In [177]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100,
) -> list[str]:
    """文字列とその関連性のリストを、関連性の高いものから低いものへとソートして返す。"""
    query_embedding_response = embedding_request(query)
    query_embedding = query_embedding_response["data"][0]["embedding"]
    strings_and_relatednesses = [
        (row["filepath"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n]

In [178]:
def read_pdf(filepath):
    """PDFへのファイルパスを受け取り、PDFの内容の文字列を返す。"""
    # pdfリーダーオブジェクトの作成
    reader = PdfReader(filepath)
    pdf_text = ""
    page_number = 0
    for page in reader.pages:
        page_number += 1
        pdf_text += page.extract_text() + f"\nPage Number: {page_number}"
    return pdf_text


# テキストをサイズnの小さな塊に分割し、できれば文末で終わるようにする
def create_chunks(text, n, tokenizer):
    """指定されたテキストから連続したn個のチャンクを返します。"""
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # 0.5*n個と1.5*n個のトークンの範囲内で、最も近い文末を探す
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # トークンをデコードし、フルストップや改行があるかどうかをチェックする。
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # 文末が見つからない場合、チャンクサイズとしてn個のトークンを使用する
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j


def extract_chunk(content, template_prompt):
    """この関数は、入力されたコンテンツにプロンプトを適用する。この場合、要約されたテキストの塊が返されます。"""
    prompt = template_prompt + content
    response = openai.ChatCompletion.create(
        model=GPT_MODEL, messages=[{"role": "user", "content": prompt}], temperature=0
    )
    return response["choices"][0]["message"]["content"]


def summarize_text(query):
    """この関数は、以下の処理を行います：
    - 埋め込みを含むarxiv_library.csvファイルを読み込む。
    - ユーザーのクエリに最も近いファイルを検索します。
    - ファイルからテキストを取り出し、チャンク化する。
    - 各チャンクを並行して要約する
    - 最終的に1つの要約を行い、ユーザーに返す"""

    # 再帰的要約が入力論文にどのようにアプローチすべきかを指示するためのプロンプト
    summary_prompt = """学術論文からこのテキストを要約しなさい。重要な点があれば、理由をつけて抜き出す\n\nContent:"""

    # ライブラリが空の場合（まだ検索が行われていない場合）、検索を行い、結果をダウンロードします
    library_df = pd.read_csv(paper_dir_filepath).reset_index()
    if len(library_df) == 0:
        print("まだ検索された論文はない、まずはダウンロードを。")
        get_articles(query)
        print("論文ダウンロード…継続中")
        library_df = pd.read_csv(paper_dir_filepath).reset_index()
    library_df.columns = ["title", "filepath", "embedding"]
    library_df["embedding"] = library_df["embedding"].apply(ast.literal_eval)
    strings = strings_ranked_by_relatedness(query, library_df, top_n=1)
    print("Chunking text from paper")
    pdf_text = read_pdf(strings[0])

    # Initialise tokenizer
    tokenizer = tiktoken.get_encoding("cl100k_base")
    results = ""

    # ドキュメントを1500トークンのチャンクに切り分ける
    chunks = create_chunks(pdf_text, 1500, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    print("Summarizing each chunk of text")

    # サマリーを並列処理する
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=len(text_chunks)
    ) as executor:
        futures = [
            executor.submit(extract_chunk, chunk, summary_prompt)
            for chunk in text_chunks
        ]
        with tqdm(total=len(text_chunks)) as pbar:
            for _ in concurrent.futures.as_completed(futures):
                pbar.update(1)
        for future in futures:
            data = future.result()
            results += data

    # 最終まとめ
    print("全体まとめにまとめる")
    response = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[
            {
                "role": "user",
                "content": f"""Write a summary collated from this collection of key points extracted from an academic paper.
                        The summary should highlight the core argument, conclusions and evidence, and answer the user's query.  lang:ja
                        User query: {query}
                        The summary should be structured in bulleted lists following the headings Core Argument, Evidence, and Conclusions.
                        Key points:\n{results}\nSummary:\n lang:ja""",
            }
        ],
        temperature=0,
    )
    return response

In [179]:
# summarize_text関数が動作することをテストする
## chat_test_response = summarize_text("PPO reinforcement learning sequence generation")
chat_test_response = summarize_text("computer graphics metaverse")


Chunking text from paper
Summarizing each chunk of text


100%|██████████| 5/5 [00:20<00:00,  4.03s/it]


全体まとめにまとめる


In [180]:
print(chat_test_response["choices"][0]["message"]["content"])

- この論文は、セマンティックコミュニケーションとAIによるコンテンツ生成を統合するための統一フレームワークについて述べています。
- 統合されたセマンティックコミュニケーションとAIによるコンテンツ生成（ISGC）は、セマンティック情報をユーザーの入力から転送し、デジタルコンテンツを生成し、メタバースのグラフィックをレンダリングすることによって注目を集めています。
- ISGCは、リソースの最適化された割り当てのための統合ゲインと、目標志向の高品質コンテンツ生成のための調整ゲインの2つの主な利点を提供します。
- ISGCは、セマンティック抽出、コンテンツ生成、グラフィックレンダリングのための最適なリソース割り当て戦略を特定するための拡散モデルを使用したケーススタディを提供します。
- ISGCの潜在的な応用についても議論されています。
- ISGCのフレームワークは、Semantic Module、Inference Module、Rendering Moduleから構成されています。
- ISGCの主な利点は、統合による利益であり、シームレスに連携するコンポーネントやリソースを提供します。
- ISGCの特徴は、リソースの効率的な利用、柔軟性の向上、統合による利益の提供です。
- ISGCの将来の方向性についても議論されており、メタバースでの使用におけるいくつかの課題と方向性が提案されています。


## エージェントの設定
ここでは、arXivデータへのアクセスを提供する関数について、2つの関数仕様を作成します。また、Chat Completions API 呼び出しと関数実行を統合するためのユーティリティもいくつか作成する予定です。

In [181]:
# get_articles と read_article_and_summarize 関数を開始する。
arxiv_functions = [
    {
        "name": "get_articles",
        "description": """ユーザーの質問に答えるために、arXivから学術論文を取得するために使用する機能です。""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            User query in JSON. 回答は要約し、記事URLの参照を含める必要があります。
                            """,
                }
            },
            "required": ["query"],
        },
        "name": "read_article_and_summarize",
        "description": """論文全体を読み、ユーザー向けに要約を提供する機能です。
        会話でget_articlesが呼び出される前にこの関数を呼び出すことは 絶対に しないでください。""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            ユーザーのクエリに基づくプレーンテキストでの記事の説明
                            """,
                }
            },
            "required": ["query"],
        },
    }
]

In [182]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """この関数は、ChatCompletion APIを呼び出すもので、オプションで関数を追加することができます。"""
    response = chat_completion_request(messages, functions)
    full_message = response.json()["choices"][0]
    if full_message["finish_reason"] == "function_call":
        print(f"Function generation requested, calling function")
        return call_arxiv_function(messages, full_message)
    else:
        print(f"Function not required, responding to user")
        return response.json()


def call_arxiv_function(messages, full_message):
    """モデルが必要と判断した場合に、関数呼び出しを実行する関数呼び出し機能。
    現在は、このif文に節を追加して拡張しています。"""

    if full_message["message"]["function_call"]["name"] == "get_articles":
        try:
            parsed_output = json.loads(
                full_message["message"]["function_call"]["arguments"]
            )
            print("Getting search results")
            results = get_articles(parsed_output["query"])
        except Exception as e:
            print(parsed_output)
            print(f"Function execution failed")
            print(f"Error message: {e}")
        messages.append(
            {
                "role": "function",
                "name": full_message["message"]["function_call"]["name"],
                "content": str(results),
            }
        )
        try:
            print("検索結果を取得し、コンテンツを要約する")
            response = chat_completion_request(messages)
            return response.json()
        except Exception as e:
            print(type(e))
            raise Exception("Function chat request failed")

    elif (
        full_message["message"]["function_call"]["name"] == "read_article_and_summarize"
    ):
        parsed_output = json.loads(
            full_message["message"]["function_call"]["arguments"]
        )
        print("Finding and reading paper")
        summary = summarize_text(parsed_output["query"])
        return summary

    else:
        raise Exception("機能が存在しないため、呼び出すことができない")

## arXiv conversation
Let's test out our function in conversation

In [183]:
# システムメッセージから始める
paper_system_message = """あなたはarXivGPT、ユーザの質問に答えるために、論文を引用するアシスタント。
あなたは論文を明確に要約し、ユーザが自分の質問に答えるためにどれを読むべきかを判断できるようにします。
ユーザーが論文を理解し、アクセスできるように、常に記事のURLとタイトルを提供します。
さあ始めてください！"""
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)
# ユーザーメッセージの追加
paper_conversation.add_message("user", "こんにちは、PPO強化学習とはどのようなものなのでしょうか？")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))

Function generation requested, calling function
Finding and reading paper
Chunking text from paper
Summarizing each chunk of text


100%|██████████| 10/10 [00:16<00:00,  1.61s/it]


全体まとめにまとめる


InvalidRequestError: ignored

In [ ]:
# Add another user message to induce our system to use the second tool
paper_conversation.add_message(
    "user",
    "Can you read the PPO sequence generation paper for me and give me a summary",
)
updated_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)
display(Markdown(updated_response["choices"][0]["message"]["content"]))